In [23]:
# import  cac model hoi quy
from sklearn.linear_model import LinearRegression 
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import PolynomialFeatures , OneHotEncoder , RobustScaler
from sklearn.tree import DecisionTreeRegressor 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error



dataset = pd.read_csv('D:\jupyter\LandTrading.csv')
df = dataset[dataset['ten_quan'] == 'Huyện Hoài Đức']
df1 = df[['dien_tich','mat_tien','so_do','lat','long','gia','gia_m2','do_rong_duong','do_rong_duong_ml']]
df2 = df[['dien_tich','mat_tien','so_do','lat','long','gia','gia_m2','do_rong_duong','do_rong_duong_ml']]
df1 = df1.reset_index(drop=True)

# xử lí dữ liệu khuyết thiếu

df1['so_do'] = df1['so_do'].fillna('Không có')
df1['mat_tien'] = df1['mat_tien'].fillna(0)
df1['do_rong_duong'] = df1['do_rong_duong'].fillna(0)
df1['do_rong_duong_ml'] = df1['do_rong_duong_ml'].fillna(0)

def impute_mean(series):
    return series.fillna(series.mean())
df1['lat'] = df1['lat'].transform(impute_mean)
df1['long'] = df1['long'].transform(impute_mean)


def do_rong_duong (ml):
    try:
        ml = ml.lower()
        if ml == 'ngõ ngách':
            return 1
        elif ml=='ngõ 1 ô tô':
            return 2
        elif ml == 'ngõ 2 ô tô tránh':
            return 3
        elif ml == 'ngõ 3 ô tô tránh':
            return 4
        elif ml == 'ngõ 4 ô tô tránh':
            return 5
        elif ml == 'ngõ 4 ô tô trở lên':
            return 6
        
    except:
        return 0
df1['do_rong_duong_ml'] = df1['do_rong_duong_ml'].transform(do_rong_duong)


def sodo(value):
    if value=='Không có':
        return 0
    else:
        return 1
df1['so_do'] = df1['so_do'].transform(sodo)


df1 = df1.dropna(subset = ['dien_tich','gia','gia_m2'],thresh = 2)
# # sns.boxplot(df1['gia'])
def impute_median(value):
     return value.fillna(value.median())
df1['gia'] = df1['gia'].transform(impute_median)
df1['gia_m2'] = df1['gia_m2'].transform(impute_median)
# df1 = df1.dropna(axis='rows')


# sns.heatmap(df1.corr(),vmin=0,vmax=1,annot=True,cmap=plt.cm.Reds)
# plt.show()



# ======XỬ LÍ DỮ LIỆU NGOẠI LAI===========
df5 = df1[['gia','gia_m2','dien_tich']]
Q1 = df5.quantile(0.25)
Q3 = df5.quantile(0.75)
IQR = Q3 - Q1


df5['outlier']= ~((df5 < (Q1 - 1.5 * IQR))|(df5 > (Q3 + 1.5 * IQR))).any(axis=1)
df5 = df5[df5['outlier'] == True]
# df5.rename(columns = {'dien_tich':'S','gia':'gia 2','gia_m2':'gia_m2 2'}, inplace = True)
# df5.to_csv('df5 test.csv')
del df1['dien_tich'] , df1['gia'] , df1['gia_m2']
df2 = pd.concat([df1, df5.reindex(df1.index)], axis=1)
del df2['outlier']
df2 = df2.dropna(axis='rows')

# print(df2.describe())

# sns.heatmap(df2.corr(),vmin=0,vmax=1,annot=True,cmap=plt.cm.Blues)
# plt.show()



# 2
# scaler = RobustScaler()
# # Chuẩn hóa dữ liệu trong df với StandardScaler
# df_s = scaler.fit_transform(df2)

# # lấy danh sách cột
# col_names = list(df2.columns)

# # chuyển về DataFrame, gán các cột của df cho dữ liệu đã được chuẩn hóa
# df_s = pd.DataFrame(df_s, columns=col_names)
# sns.boxplot(data=df_s)
# plt.show()


# ============= HỒI QUI TUYẾN TÍNH ================
X = df2.drop('gia_m2', axis = 1)
Y =df2['gia_m2']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)
# regressor = DecisionTreeRegressor()
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
pred = regressor.predict(X_test)
print("MAPE: ",mean_absolute_percentage_error(Y_test, pred))
# tính toán R2 của model
import sklearn.metrics as metrics
from sklearn.metrics import r2_score
r2_train = r2_score(Y_train, regressor.predict(X_train))
print("R2 trên tập huấn luyện của model là:"  + str(r2_train))
r2_test = r2_score(Y_test, pred)
print("R2 trên tập kiểm tra của model là:"  + str(r2_test))
print('Sai số dự báo trung bình:', metrics.mean_absolute_error(Y_test, pred)) 
# toi co mot can nha vi tri lat long, va co dien tich 55m2, thi hoi rang la nha do bao nhieu tien
# theo dung thu tu
input = [5,1,35,120,5,5,1530.5,170]
input = np.array(input).reshape(1,-1)
estimate = regressor.predict(input)
print(estimate)
# print(df1.describe())



























(336, 8)
(786, 8)
(336,)
(786,)
MAPE:  1.151148001464019
R2 trên tập huấn luyện của model là:0.8596644020390846
R2 trên tập kiểm tra của model là:0.780501960467779
Sai số dự báo trung bình: 4.010281203514578
[456.76510595]


c:\Users\Viet Anh\Anaconda3\lib\site-packages\ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\Users\Viet Anh\Anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:

     
   
from sklearn.linear_model import LinearRegression
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

df = pd.read_csv('D:\jupyter\LandTrading.csv')
df1 = df[df['ten_quan']=='Huyện Hoài Đức']
df1['so_do']

df1['so_do'] = df1['so_do'].fillna('Không')
def convert_land(land):
    try:
        land = land.lower()
        if _found_text(land,['sổ đỏ'],['đã có sổ'],['sđcc']):
            return 2
        if _found_text(land,['không']):
            return 0
        else:
            return 1
def _found_text(text, list_item):
# check whether the text contains any item within the list_item
for item in list_item:
    if item in text:
        return True
return False

df1['so_do'] = df1['so_do'].transform(convert_land)
